In [150]:
import pandas as pd
from sklearn.metrics.pairwise import sigmoid_kernel
from sklearn.metrics.pairwise import cosine_similarity
from sklearn import preprocessing

df=pd.read_csv("data 2.csv")

df.head()

,Unnamed: 0,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,target,song_title,artist
0,0,0.0102,0.833,204600,0.434,0.021900,2,0.1650,-8.795,1,0.4310,150.062,4.0,0.286,1,Mask Off,Future
1,1,0.1990,0.743,326933,0.359,0.006110,1,0.1370,-10.401,1,0.0794,160.083,4.0,0.588,1,Redbone,Childish Gambino
2,2,0.0344,0.838,185707,0.412,0.000234,2,0.1590,-7.148,1,0.2890,75.044,4.0,0.173,1,Xanny Family,Future
3,3,0.6040,0.494,199413,0.338,0.510000,5,0.0922,-15.236,1,0.0261,86.468,4.0,0.230,1,Master Of None,Beach House
4,4,0.1800,0.678,392893,0.561,0.512000,5,0.4390,-11.648,0,0.0694,174.004,4.0,0.904,1,Parallel Lines,Junior Boys


In [151]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2017 entries, 0 to 2016
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Unnamed: 0        2017 non-null   int64  
 1   acousticness      2017 non-null   float64
 2   danceability      2017 non-null   float64
 3   duration_ms       2017 non-null   int64  
 4   energy            2017 non-null   float64
 5   instrumentalness  2017 non-null   float64
 6   key               2017 non-null   int64  
 7   liveness          2017 non-null   float64
 8   loudness          2017 non-null   float64
 9   mode              2017 non-null   int64  
 10  speechiness       2017 non-null   float64
 11  tempo             2017 non-null   float64
 12  time_signature    2017 non-null   float64
 13  valence           2017 non-null   float64
 14  target            2017 non-null   int64  
 15  song_title        2017 non-null   object 
 16  artist            2017 non-null   object 


In [152]:
feature_cols=['acousticness', 'danceability', 'duration_ms', 'energy',
              'instrumentalness', 'key', 'liveness', 'loudness', 'mode',
              'speechiness', 'tempo', 'time_signature', 'valence',]

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler() #normalization of features
normalized_df =scaler.fit_transform(df[feature_cols])

print(normalized_df[:2])

[[0.01024843 0.82482599 0.19073524 0.4263629  0.02243852 0.18181818
  0.15386234 0.74114059 1.         0.51444066 0.59603317 0.75
  0.26243209]
 [0.19999772 0.72041763 0.3144808  0.35008137 0.00626025 0.09090909
  0.12439486 0.69216224 1.         0.07100517 0.6544742  0.75
  0.57793565]]


In [154]:
# Create a pandas series with song titles as indices and indices as values
indices = pd.Series(df.index, index=df['song_title']).drop_duplicates()

print(indices)

# Create cosine similarity matrix based on given normalized df
cosine = cosine_similarity(normalized_df)
print(cosine)

def generate_recommendation(song_title, model_type=cosine ):
    """
    Purpose: Function for song recommendations
    Inputs: song title and type of similarity model
    Output: Pandas series of recommended songs
    """
    # Get song indices
    index=indices[song_title]
    # Get list of songs for given songs
    score=list(enumerate(model_type[indices[song_title]]))
    # Sort the most similar songs
    similarity_score = sorted(score,key = lambda x:x[1],reverse = True)
    # Select the top-10 recommend songs
    similarity_score = similarity_score[1:11]
    top_songs_index = [i[0] for i in similarity_score]
    # Top 10 recommende songs
    top_songs = df.loc[top_songs_index, ['song_title', 'artist']]
    return top_songs

song_title
Mask Off                                   0
Redbone                                    1
Xanny Family                               2
Master Of None                             3
Parallel Lines                             4
                                        ... 
Like A Bitch - Kill The Noise Remix     2012
Candy                                   2013
Habit - Dack Janiels & Wenzday Remix    2014
First Contact                           2015
I Wanna Get Better                      2016
Length: 2017, dtype: int64
[[1.         0.94869421 0.96780513 ... 0.92940452 0.87132508 0.88341223]
 [0.94869421 1.         0.91619328 ... 0.94037467 0.88680786 0.87627884]
 [0.96780513 0.91619328 1.         ... 0.91176001 0.84476836 0.83986284]
 ...
 [0.92940452 0.94037467 0.91176001 ... 1.         0.94382664 0.91458893]
 [0.87132508 0.88680786 0.84476836 ... 0.94382664 1.         0.88017332]
 [0.88341223 0.87627884 0.83986284 ... 0.91458893 0.88017332 1.        ]]


In [155]:
print("Recommended Songs:")
print(generate_recommendation('Cha Cha Slide',cosine).values) #Put a song title from the csv here
#Unfortunently, Songs that have the same name in the csv won't work sadly

Recommended Songs:
[['Got To Give It Up (Part 1)' 'Marvin Gaye']
 ['The Fighter' 'Keith Urban']
 ['The Call' 'Backstreet Boys']
 ['Light It Up (feat. Nyla & Fuse ODG) [Remix]' 'Major Lazer']
 ['212' 'Azealia Banks']
 ['White Noise' 'Disclosure']
 ['Razor' 'ASTR']
 ['Quit Playing Games (With My Heart)' 'Backstreet Boys']
 ["Everybody (Backstreet's Back) - Radio Edit" 'Backstreet Boys']
 ['Saltkin' 'Purity Ring']]
